In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from PIL import Image, ImageEnhance
from glob import glob
from matplotlib import cm
import cv2
import numpy as np

**Low-light**

In [ ]:

i =0 
#read the image
for filename in glob.glob('/content/drive/MyDrive/dataset-combination/combination2/test-n/*.jpg'):
  #print(filename)
   i+= 1
   im = Image.open(filename)
  
#image brightness enhancer
   enhancer = ImageEnhance.Brightness(im)
   #path = '/content/drive/MyDrive/dataset-combination/combination2/darkened image/'+'PPE_'+str(i)+'.jpg'
#factor = 1 #gives original image
#im_output = enhancer.enhance(factor)
#im_output.save('original.jpg')
   t = filename.split('/')
   splitted_name = os.path.splitext(t[8])
   path = '/content/drive/MyDrive/dataset-combination/combination2/darkened image/'+splitted_name[0]+'.jpg' 
   factor = 0.4 #darkens the image
   im_output = enhancer.enhance(factor)
   im_output.save(path)
   #if (i>2):
   #  break
#factor = 1.5 #brightens the image
#im_output = enhancer.enhance(factor)
#im_output.save('brightened-image.jpg')

**Haze**

In [ ]:
def genHaze(in_img):
    max_val_1 = np.max(in_img)
    out_img = np.float64(in_img) + 100
    max_val_2 = np.max(out_img)
    out_img = np.uint8(out_img * max_val_1 / max_val_2)
    return out_img

In [ ]:
def genNoisyHaze(in_img, noise_amplitude = 50, noise_offset = 100):
    """
    noise_amplitude = Adjust this to vary the severity of noise in the haze. 50 by default
    noise_offset = Adjust this to vary the brightness of noise in the haze. 100 by default
    """
    noisy_img = noise_amplitude * np.random.rand(*in_img.shape[:2]) + noise_offset      # 2D
    noisy_img = np.stack((noisy_img, noisy_img, noisy_img), axis = 2)                   # 3D

    max_val_1 = np.max(in_img)
    out_img = np.float64(in_img) + noisy_img
    out_img = np.float64(in_img) + 250
    max_val_2 = np.max(out_img)
    out_img = np.uint8(out_img * max_val_1 / max_val_2)
    return out_img

In [ ]:

i = 0
for filename in glob('/content/drive/MyDrive/dataset-combination/combination2/test-n/*.jpg'):
   org_img = cv2.imread(filename)
   org_img = cv2.cvtColor(org_img, cv2.COLOR_BGR2RGB)
   #haze_img = genHaze(org_img)
   noisy_haze_img = genNoisyHaze(org_img, noise_amplitude = 50, noise_offset = 100)
   im = Image.fromarray(noisy_haze_img)
   t = filename.split('/')
   splitted_name = os.path.splitext(t[8])
   path = '/content/drive/MyDrive/dataset-combination/combination2/hazy-image/'+splitted_name[0]+'.jpg' 
   #im.save('/content/ab/'+splitted_name[0]+'.jpg')
   im.save(path)
  # i+=1
   #if (i>5):
   # break

**Rain**

In [ ]:
def generate_random_lines(imshape, slant, drop_length):    
    drops = []    
    for i in range(1000): ## If You want heavy rain, try increasing this        
        if slant < 0:            
            x = np.random.randint(slant, imshape[1])        
        else:            
            x = np.random.randint(0, imshape[1] - slant)        
        y = np.random.randint(0, imshape[0] - drop_length)        
        drops.append((x, y))    
    return drops
        
def add_rain(image):
    image = image.copy()
    imshape = image.shape    
    slant_extreme = 10    
    slant = np.random.randint(-slant_extreme, slant_extreme)     
    drop_length = int(imshape[0] * .02)
    drop_width = int(imshape[1] * .0018)
    if (drop_width == 0):
       drop_width = 1
   # print(drop_length)
   # print(drop_width)

    #drop_length = 20
    #drop_width = 5
    drop_color = (196, 211, 223) ## a shade of gray

    rain_drops = generate_random_lines(imshape, slant, drop_length)        
    for rain_drop in rain_drops:        
        cv2.line(image, (rain_drop[0], rain_drop[1]), (rain_drop[0] + slant, rain_drop[1] + drop_length), drop_color, drop_width)    
    
    # kernelSize = min(imshape[:2])
    # kernelSize = int(kernelSize * .0125)
    # if kernelSize % 2 == 0: kernelSize += 1
    # if kernelSize < 3: kernelSize = 3
    kernelSize = 4
    
    image = cv2.blur(image, (kernelSize, kernelSize))     ## rainy view are blurry        
    brightness_coefficient = 0.7      ## rainy days are usually shady     
    image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)      ## Conversion to HLS    
    image_HLS[:, :, 1] = image_HLS[:, :, 1] * brightness_coefficient    ## scale pixel values down for channel 1(Lightness)    
    image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB)  ## Conversion to RGB    
    return image

In [ ]:

i = 0
for filename in glob('/content/drive/MyDrive/dataset-combination/combination2/test-n/*.jpg'):
   org_img = cv2.imread(filename)
   org_img = cv2.cvtColor(org_img, cv2.COLOR_BGR2RGB)
   #haze_img = genHaze(org_img)
   rainy_img = add_rain(org_img)
   im = Image.fromarray(rainy_img)
   t = filename.split('/')
   splitted_name = os.path.splitext(t[8])
   path = '/content/drive/MyDrive/dataset-combination/combination2/rainy-image/'+splitted_name[0]+'.jpg' 
   #im.save('/content/'+splitted_name[0]+'.jpg')
   im.save(path)
  # i+=1
 #  if (i>3):
  #  break

In [ ]:

i = 0 
for filename in glob.glob('/content/drive/MyDrive/dataset-combination/combination2/test-n/*.jpg'):
    print(filename)
    t = filename.split('/')
    splitted_name = os.path.splitext(t[8])
    print(splitted_name[0])
    i+=1
    if (i>2):
      break